<a href="https://colab.research.google.com/github/hiddenntreasure/Har/blob/master/HAR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
s=pd.Series([1,2,3,4,5])
s=s.as_matrix()
t=[]
o=pd.Series([6,7,8,9,10])
o=o.as_matrix()
t.append(s)
print(t)
t.append(o)
print(t)
t

[array([1, 2, 3, 4, 5])]
[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]

In [0]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

#DATA

In [0]:
DATADIR = '/content/drive/My Drive/Colab Notebooks/CS 10: Human Acitivity Recognition/UCI_HAR_Dataset'

In [0]:
SIGNALS = [
           "body_acc_x",
           "body_acc_y",
           "body_acc_z",
           "body_gyro_x",
           "body_gyro_y",
           "body_gyro_z",
           "total_acc_x",
           "total_acc_y",
           "total_acc_z"
]

In [0]:
def _read_csv(filename):
  return pd.read_csv(filename,delim_whitespace=True,Header=None)

def load_signals(subset):
  signals_data=[]
  for signal in SIGNALS:
    filename = f'{DATADIR}/{subset}/Inertial Signalls/{signal}_{subset}.txt'
    signals_data.append(
        _read_csv(filename).as_matrix()
    )

    return np.transpose(signals_data,(1,2,0))